# Libraries & Data

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn import metrics
from time import time

# DT & RF
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import tree
from statsmodels.stats.outliers_influence import variance_inflation_factor
# XGBoost
from xgboost import XGBClassifier
#LogReg
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import statsmodels.tools
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

with open('ufc_model.pkl', 'rb') as f:
    best = pickle.load(f)
df = pd.read_csv("UFC.csv")
df.head()

,event_id,event_name,date,location,fight_id,division,title_fight,method,finish_round,match_time_sec,total_rounds,referee,r_name,r_id,r_kd,r_sig_str_landed,r_sig_str_atmpted,r_sig_str_acc,r_total_str_landed,r_total_str_atmpted,r_total_str_acc,r_td_landed,r_td_atmpted,r_td_acc,r_sub_att,r_ctrl,r_head_landed,r_head_atmpted,r_head_acc,r_body_landed,r_body_atmpted,r_body_acc,r_leg_landed,r_leg_atmpted,r_leg_acc,r_dist_landed,r_dist_atmpted,r_dist_acc,r_clinch_landed,r_clinch_atmpted,r_clinch_acc,r_ground_landed,r_ground_atmpted,r_ground_acc,r_landed_head_per,r_landed_body_per,r_landed_leg_per,r_landed_dist_per,r_landed_clinch_per,r_landed_ground_per,r_nick_name,r_wins,r_losses,r_draws,r_height,r_weight,r_reach,r_stance,r_dob,r_splm,r_str_acc,r_sapm,r_str_def,r_td_avg,r_td_avg_acc,r_td_def,r_sub_avg,b_name,b_id,b_kd,b_sig_str_landed,b_sig_str_atmpted,b_sig_str_acc,b_total_str_landed,b_total_str_atmpted,b_total_str_acc,b_td_landed,b_td_atmpted,b_td_acc,b_sub_att,b_ctrl,b_head_landed,b_head_atmpted,b_head_acc,b_body_landed,b_body_atmpted,b_body_acc,b_leg_landed,b_leg_atmpted,b_leg_acc,b_dist_landed,b_dist_atmpted,b_dist_acc,b_clinch_landed,b_clinch_atmpted,b_clinch_acc,b_ground_landed,b_ground_atmpted,b_ground_acc,b_landed_head_per,b_landed_body_per,b_landed_leg_per,b_landed_dist_per,b_landed_clinch_per,b_landed_ground_per,b_nick_name,b_wins,b_losses,b_draws,b_height,b_weight,b_reach,b_stance,b_dob,b_splm,b_str_acc,b_sapm,b_str_def,b_td_avg,b_td_avg_acc,b_td_def,b_sub_avg,winner,winner_id
0,6e380a4d73ab4f0e,UFC Fight Night: Imavov vs. Borralho,2025/09/06,"Paris, Ile-de-France, France",d14fea43712707f0,lightweight,0,Submission,2,176,3.0,Herb Dean,Benoit Saint Denis,c2299ec916bc7c56,0.0,17.0,38.0,44.0,37.0,61.0,61.0,3.0,10.0,30.0,1.0,278.0,9.0,26.0,35.0,8.0,12.0,67.0,0.0,0.0,NaN,9.0,26.0,35.0,3.0,3.0,100.0,5.0,9.0,56.0,52.0,47.0,0.0,52.0,17.0,29.0,God of War,15,3,0,180.34,70.31,185.42,Southpaw,1995/12/18,5.05,55,4.19,42,4.35,37,70,1.6,Mauricio Ruffy,9c393e836a852f30,0.0,5.0,17.0,29.0,6.0,18.0,33.0,0.0,0.0,NaN,0.0,34.0,4.0,14.0,28.0,1.0,3.0,33.0,0.0,0.0,NaN,5.0,17.0,28.0,0.0,0.0,NaN,0.0,0.0,NaN,80.0,20.0,0.0,100.0,0.0,0.0,NaN,12,2,0,180.34,70.31,190.50,Orthodox,1996/06/17,3.84,57,4.16,60,0.00,0,84,0.0,Benoit Saint Denis,c2299ec916bc7c56
1,6e380a4d73ab4f0e,UFC Fight Night: Imavov vs. Borralho,2025/09/06,"Paris, Ile-de-France, France",5bcd8abf6da01dae,middleweight,0,Decision - Unanimous,5,300,5.0,Marc Goddard,Nassourdine Imavov,881bf86d4cba8578,0.0,81.0,162.0,50.0,89.0,170.0,52.0,0.0,0.0,NaN,0.0,29.0,53.0,118.0,45.0,14.0,26.0,54.0,14.0,18.0,78.0,79.0,160.0,49.0,2.0,2.0,100.0,0.0,0.0,NaN,65.0,17.0,17.0,97.0,2.0,0.0,The Sniper,17,4,0,190.50,83.91,190.50,Orthodox,1996/03/01,4.28,54,3.17,58,0.74,32,81,1.0,Caio Borralho,4126a78111c0855a,0.0,66.0,166.0,39.0,89.0,190.0,47.0,0.0,5.0,0.0,0.0,80.0,39.0,109.0,36.0,14.0,38.0,37.0,13.0,19.0,68.0,59.0,159.0,37.0,7.0,7.0,100.0,0.0,0.0,NaN,59.0,21.0,19.0,89.0,10.0,0.0,The Natural,17,2,0,185.42,83.91,190.50,Southpaw,1993/01/16,3.44,56,2.50,60,1.28,48,76,0.4,Nassourdine Imavov,881bf86d4cba8578
2,6e380a4d73ab4f0e,UFC Fight Night: Imavov vs. Borralho,2025/09/06,"Paris, Ile-de-France, France",0149e875b31e57f8,light heavyweight,0,KO/TKO,1,300,3.0,Marc Goddard,Modestas Bukauskas,476fe566d2df676e,0.0,22.0,36.0,61.0,29.0,44.0,66.0,0.0,0.0,NaN,0.0,110.0,18.0,30.0,60.0,0.0,0.0,NaN,4.0,6.0,67.0,10.0,17.0,59.0,0.0,0.0,NaN,12.0,19.0,63.0,81.0,0.0,18.0,45.0,0.0,54.0,The Baltic Gladiator,19,6,0,190.50,92.99,193.04,Switch,1994/02/10,3.32,43,3.93,51,0.30,66,80,0.1,Paul Craig,eabf206b162b3b83,0.0,6.0,14.0,42.0,30.0,39.0,77.0,0.0,2.0,0.0,0.0,45.0,0.0,4.0,0.0,2.0,3.0,67.0,4.0,7.0,56.0,5.0,13.0,38.0,1.0,1.0,100.0,0.0,0.0,NaN,0.0,33.0,66.0,83.0,16.0,0.0,Bearjew,17,10,1,190.50,92.99,193.04,Orthodox,1987/11/27,2.50,45,3.04,43,1.43,18,35,1.3,Modestas Bukauskas,476fe566d2df676e
3,6e380a4d73ab4f0e,UFC Fight Night: Imavov vs. Borralho,2025/09/06,"Paris, Ile-de-France, France",69b31593175f77ec,lightweight,0,KO/TKO,2,198,3.0,Rich Mitch

SLpM - Significant Strikes Landed per Minute

Str. Acc. - Significant Striking Accuracy

SApM - Significant Strikes Absorbed per Minute

Str. Def. - Significant Strike Defence (the % of opponents strikes that did not land)

TD Avg. - Average Takedowns Landed per 15 minutes

TD Acc. - Takedown Accuracy

TD Def. - Takedown Defense (the % of opponents TD attempts that did not land)

Sub. Avg. - Average Submissions Attempted per 15 minutes

# EDA

In [2]:
df.shape

(8337, 124)

In [3]:
df.dtypes

event_id                object
event_name              object
date                    object
location                object
fight_id                object
division                object
title_fight              int64
method                  object
finish_round             int64
match_time_sec           int64
total_rounds           float64
referee                 object
r_name                  object
r_id                    object
r_kd                   float64
r_sig_str_landed       float64
r_sig_str_atmpted      float64
r_sig_str_acc          float64
r_total_str_landed     float64
r_total_str_atmpted    float64
r_total_str_acc        float64
r_td_landed            float64
r_td_atmpted           float64
r_td_acc               float64
r_sub_att              float64
r_ctrl                 float64
r_head_landed          float64
r_head_atmpted         float64
r_head_acc             float64
r_body_landed          float64
r_body_atmpted         float64
r_body_acc             float64
r_leg_la

In [4]:
(df.isnull().sum() / df.shape[0]) * 100

event_id                0.000000
event_name              0.000000
date                    0.000000
location                0.000000
fight_id                0.000000
division                0.000000
title_fight             0.000000
method                  0.000000
finish_round            0.000000
match_time_sec          0.000000
total_rounds            0.371836
referee                 0.311863
r_name                  0.000000
r_id                    0.000000
r_kd                    0.251889
r_sig_str_landed        0.251889
r_sig_str_atmpted       0.251889
r_sig_str_acc           0.707689
r_total_str_landed      0.251889
r_total_str_atmpted     0.251889
r_total_str_acc         0.575747
r_td_landed             0.251889
r_td_atmpted            0.251889
r_td_acc               32.217824
r_sub_att               0.251889
r_ctrl                  2.422934
r_head_landed           0.251889
r_head_atmpted          0.251889
r_head_acc              1.187478
r_body_landed           0.251889
r_body_atm

Some columns will need to be filled in with a mean/median due to a high percentage of nulls
* To be done after train/test split due to data leakage
* Winners have some nulls, drop rows?

In [6]:
df.r_stance.unique() # Potential OHE

array(['Southpaw', 'Orthodox', 'Switch', nan, 'Open Stance', 'Sideways'],
      dtype=object)

# Cleaning

* Creating rolling avg of all per fight stats

In [7]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")

per_fight_cols = [
    "kd",
    "sig_str_landed",
    "sig_str_atmpted",
    "sig_str_acc",
    "total_str_landed",
    "total_str_atmpted",
    "total_str_acc",
    "td_landed",
    "td_atmpted",
    "td_acc",
    "sub_att",
    "ctrl",
    "head_landed",
    "head_atmpted",
    "head_acc",
    "body_landed",
    "body_atmpted",
    "body_acc",
    "leg_landed",
    "leg_atmpted",
    "leg_acc",
    "dist_landed",
    "dist_atmpted",
    "dist_acc",
    "clinch_landed",
    "clinch_atmpted",
    "clinch_acc",
    "ground_landed",
    "ground_atmpted",
    "ground_acc",
    "landed_head_per",
    "landed_body_per",
    "landed_leg_per",
    "landed_dist_per",
    "landed_clinch_per",
    "landed_ground_per",
]

# Map red - base column names
r_cols = {"r_id": "fighter_id", "date": "date"}
for col in per_fight_cols:
    r_cols[f"r_{col}"] = col

red = df[["r_id", "date"] + [f"r_{c}" for c in per_fight_cols]].rename(columns=r_cols)

# Map blue - base column names
b_cols = {"b_id": "fighter_id", "date": "date"}
for col in per_fight_cols:
    b_cols[f"b_{col}"] = col

blue = df[["b_id", "date"] + [f"b_{c}" for c in per_fight_cols]].rename(columns=b_cols)

# Stack rows
fighters = pd.concat([red, blue], ignore_index=True)
fighters[per_fight_cols] = fighters[per_fight_cols].fillna(0)

# 4. Sort by fighter + date
fighters = fighters.sort_values(["fighter_id", "date"])

# 5. Compute rolling averages 
for col in per_fight_cols:
    fighters[f"roll_{col}"] = (
        fighters.groupby("fighter_id")[col]
        .transform(lambda s: s.shift(1).expanding().mean())
    )

# 6. Replace NaN with 0
rolling_cols = [f"roll_{c}" for c in per_fight_cols]
fighters[rolling_cols] = fighters[rolling_cols].fillna(0)

In [8]:
example_fighter = fighters["fighter_id"].iloc[5]
display(
    fighters[fighters["fighter_id"] == example_fighter][
        ["fighter_id", "date", "sig_str_landed", "roll_sig_str_landed"]
    ].head(10)
)

,fighter_id,date,sig_str_landed,roll_sig_str_landed
12587,0052de90691d4a93,2017-03-11,35.0,0.000000
12235,0052de90691d4a93,2017-12-09,55.0,35.000000
3722,0052de90691d4a93,2018-05-12,6.0,45.000000
3485,0052de90691d4a93,2018-11-10,4.0,32.000000
3199,0052de90691d4a93,2019-05-18,63.0,25.000000
11364,0052de90691d4a93,2019-09-07,7.0,32.600000
2668,0052de90691d4a93,2020-07-18,33.0,28.333333


In [9]:
fighters = fighters.drop_duplicates(subset=["fighter_id", "date"])
rolling_cols = ["fighter_id", "date"] + [f"roll_{c}" for c in per_fight_cols]
fighters_roll = fighters[rolling_cols].copy()

# Merge for red fighters
df = df.merge(
    fighters_roll.add_prefix("r_"),
    left_on=["r_id", "date"],
    right_on=["r_fighter_id", "r_date"],
    how="left",
    validate="many_to_one" 
)
df = df.merge(
    fighters_roll.add_prefix("b_"),
    left_on=["b_id", "date"],
    right_on=["b_fighter_id", "b_date"],
    how="left",
    validate="many_to_one"
)
df = df.drop(columns=[
    col for col in df.columns
    if col.startswith("r_fighter_id") or col.startswith("b_fighter_id")
    or col.startswith("r_date") or col.startswith("b_date")
])
print(df.shape)
df.head()

(8337, 196)


,event_id,event_name,date,location,fight_id,division,title_fight,method,finish_round,match_time_sec,total_rounds,referee,r_name,r_id,r_kd,r_sig_str_landed,r_sig_str_atmpted,r_sig_str_acc,r_total_str_landed,r_total_str_atmpted,r_total_str_acc,r_td_landed,r_td_atmpted,r_td_acc,r_sub_att,r_ctrl,r_head_landed,r_head_atmpted,r_head_acc,r_body_landed,r_body_atmpted,r_body_acc,r_leg_landed,r_leg_atmpted,r_leg_acc,r_dist_landed,r_dist_atmpted,r_dist_acc,r_clinch_landed,r_clinch_atmpted,r_clinch_acc,r_ground_landed,r_ground_atmpted,r_ground_acc,r_landed_head_per,r_landed_body_per,r_landed_leg_per,r_landed_dist_per,r_landed_clinch_per,r_landed_ground_per,r_nick_name,r_wins,r_losses,r_draws,r_height,r_weight,r_reach,r_stance,r_dob,r_splm,r_str_acc,r_sapm,r_str_def,r_td_avg,r_td_avg_acc,r_td_def,r_sub_avg,b_name,b_id,b_kd,b_sig_str_landed,b_sig_str_atmpted,b_sig_str_acc,b_total_str_landed,b_total_str_atmpted,b_total_str_acc,b_td_landed,b_td_atmpted,b_td_acc,b_sub_att,b_ctrl,b_head_landed,b_head_atmpted,b_head_acc,b_body_landed,b_body_atmpted,b_body_acc,b_leg_landed,b_leg_atmpted,b_leg_acc,b_dist_landed,b_dist_atmpted,b_dist_acc,b_clinch_landed,b_clinch_atmpted,b_clinch_acc,b_ground_landed,b_ground_atmpted,b_ground_acc,b_landed_head_per,b_landed_body_per,b_landed_leg_per,b_landed_dist_per,b_landed_clinch_per,b_landed_ground_per,b_nick_name,b_wins,b_losses,b_draws,b_height,b_weight,b_reach,b_stance,b_dob,b_splm,b_str_acc,b_sapm,b_str_def,b_td_avg,b_td_avg_acc,b_td_def,b_sub_avg,winner,winner_id,r_roll_kd,r_roll_sig_str_landed,r_roll_sig_str_atmpted,r_roll_sig_str_acc,r_roll_total_str_landed,r_roll_total_str_atmpted,r_roll_total_str_acc,r_roll_td_landed,r_roll_td_atmpted,r_roll_td_acc,r_roll_sub_att,r_roll_ctrl,r_roll_head_landed,r_roll_head_atmpted,r_roll_head_acc,r_roll_body_landed,r_roll_body_atmpted,r_roll_body_acc,r_roll_leg_landed,r_roll_leg_atmpted,r_roll_leg_acc,r_roll_dist_landed,r_roll_dist_atmpted,r_roll_dist_acc,r_roll_clinch_landed,r_roll_clinch_atmpted,r_roll_clinch_acc,r_roll_ground_landed,r_roll_ground_atmpted,r_roll_ground_acc,r_roll_landed_head_per,r_roll_landed_body_per,r_roll_landed_leg_per,r_roll_landed_dist_per,r_roll_landed_clinch_per,r_roll_landed_ground_per,b_roll_kd,b_roll_sig_str_landed,b_roll_sig_str_atmpted,b_roll_sig_str_acc,b_roll_total_str_landed,b_roll_total_str_atmpted,b_roll_total_str_acc,b_roll_td_landed,b_roll_td_atmpted,b_roll_td_acc,b_roll_sub_att,b_roll_ctrl,b_roll_head_landed,b_roll_head_atmpted,b_roll_head_acc,b_roll_body_landed,b_roll_body_atmpted,b_roll_body_acc,b_roll_leg_landed,b_roll_leg_atmpted,b_roll_leg_acc,b_roll_dist_landed,b_roll_dist_atmpted,b_roll_dist_acc,b_roll_clinch_landed,b_roll_clinch_atmpted,b_roll_clinch_acc,b_roll_ground_landed,b_roll_ground_atmpted,b_roll_ground_acc,b_roll_landed_head_per,b_roll_landed_body_per,b_roll_landed_leg_per,b_roll_landed_dist_per,b_roll_landed_clinch_per,b_roll_landed_ground_per
0,6e380a4d73ab4f0e,UFC Fight Night: Imavov vs. Borralho,2025-09-06,"Paris, Ile-de-France, France",d14fea43712707f0,lightweight,0,Submission,2,176,3.0,Herb Dean,Benoit Saint Denis,c2299ec916bc7c56,0.0,17.0,38.0,44.0,37.0,61.0,61.0,3.0,10.0,30.0,1.0,278.0,9.0,26.0,35.0,8.0,12.0,67.0,0.0,0.0,NaN,9.0,26.0,35.0,3.0,3.0,100.0,5.0,9.0,56.0,52.0,47.0,0.0,52.0,17.0,29.0,God of War,15,3,0,180.34,70.31,185.42,Southpaw,1995/12/18,5.05,55,4.19,42,4.35,37,70,1.6,Mauricio Ruffy,9c393e836a852f30,0.0,5.0,17.0,29.0,6.0,18.0,33.0,0.0,0.0,NaN,0.0,34.0,4.0,14.0,28.0,1.0,3.0,33.0,0.0,0.0,NaN,5.0,17.0,28.0,0.0,0.0,NaN,0.0,0.0,NaN,80.0,20.0,0.0,100.0,0.0,0.0,NaN,12,2,0,180.34,70.31,190.50,Orthodox,1996/06/17,3.84,57,4.16,60,0.00,0,84,0.0,Benoit Saint Denis,c2299ec916bc7c56,0.444444,40.666667,72.777778,57.777778,61.222222,97.000000,65.555556,2.111111,5.333333,39.888889,0.777778,200.222222,23.111111,51.888889,47.666667,11.777778,14.888889,76.000000,5.777778,6.000000,87.111111,23.888889,47.555556,52.333333,8.000000,9.777778,60.555556,8.777778,15.444444,54.000000,53.444444,32.444444,12.888889,63.444444,17

* Creating DF to be used for the model

In [10]:
df_model = df.copy()
df_model["target"] = (df_model["winner"] == df_model["r_name"]).astype(int)

In [11]:
df_model.drop(columns = ['event_id',
 'event_name',
 'date',
 'location',
 'fight_id',
 'division',
 'title_fight',
 'method',
 'finish_round',
 'match_time_sec',
 'total_rounds',
 'referee',
 'r_name',
 'r_id',
 'r_kd',
 'r_sig_str_landed',
 'r_sig_str_atmpted',
 'r_sig_str_acc',
 'r_total_str_landed',
 'r_total_str_atmpted',
 'r_total_str_acc',
 'r_td_landed',
 'r_td_atmpted',
 'r_td_acc',
 'r_sub_att',
 'r_ctrl',
 'r_head_landed',
 'r_head_atmpted',
 'r_head_acc',
 'r_body_landed',
 'r_body_atmpted',
 'r_body_acc',
 'r_leg_landed',
 'r_leg_atmpted',
 'r_leg_acc',
 'r_dist_landed',
 'r_dist_atmpted',
 'r_dist_acc',
 'r_clinch_landed',
 'r_clinch_atmpted',
 'r_clinch_acc',
 'r_ground_landed',
 'r_ground_atmpted',
 'r_ground_acc',
 'r_landed_head_per',
 'r_landed_body_per',
 'r_landed_leg_per',
 'r_landed_dist_per',
 'r_landed_clinch_per',
 'r_landed_ground_per',
 'r_nick_name',
 'r_wins',
 'r_losses',
 'r_draws',
 'r_dob',
 'b_name',
 'b_id',
 'b_kd',
 'b_sig_str_landed',
 'b_sig_str_atmpted',
 'b_sig_str_acc',
 'b_total_str_landed',
 'b_total_str_atmpted',
 'b_total_str_acc',
 'b_td_landed',
 'b_td_atmpted',
 'b_td_acc',
 'b_sub_att',
 'b_ctrl',
 'b_head_landed',
 'b_head_atmpted',
 'b_head_acc',
 'b_body_landed',
 'b_body_atmpted',
 'b_body_acc',
 'b_leg_landed',
 'b_leg_atmpted',
 'b_leg_acc',
 'b_dist_landed',
 'b_dist_atmpted',
 'b_dist_acc',
 'b_clinch_landed',
 'b_clinch_atmpted',
 'b_clinch_acc',
 'b_ground_landed',
 'b_ground_atmpted',
 'b_ground_acc',
 'b_landed_head_per',
 'b_landed_body_per',
 'b_landed_leg_per',
 'b_landed_dist_per',
 'b_landed_clinch_per',
 'b_landed_ground_per',
 'b_nick_name',
 'b_wins',
 'b_losses',
 'b_draws',
 'b_dob',
 'winner',
 'winner_id'] ,inplace = True)
  

In [12]:
df_model = pd.get_dummies(df_model, columns=['r_stance'], prefix='r_stance', dtype=int, drop_first=True)
df_model = pd.get_dummies(df_model, columns=['b_stance'], prefix='b_stance', dtype=int, drop_first=True)
df_model.head()

,r_height,r_weight,r_reach,r_splm,r_str_acc,r_sapm,r_str_def,r_td_avg,r_td_avg_acc,r_td_def,r_sub_avg,b_height,b_weight,b_reach,b_splm,b_str_acc,b_sapm,b_str_def,b_td_avg,b_td_avg_acc,b_td_def,b_sub_avg,r_roll_kd,r_roll_sig_str_landed,r_roll_sig_str_atmpted,r_roll_sig_str_acc,r_roll_total_str_landed,r_roll_total_str_atmpted,r_roll_total_str_acc,r_roll_td_landed,r_roll_td_atmpted,r_roll_td_acc,r_roll_sub_att,r_roll_ctrl,r_roll_head_landed,r_roll_head_atmpted,r_roll_head_acc,r_roll_body_landed,r_roll_body_atmpted,r_roll_body_acc,r_roll_leg_landed,r_roll_leg_atmpted,r_roll_leg_acc,r_roll_dist_landed,r_roll_dist_atmpted,r_roll_dist_acc,r_roll_clinch_landed,r_roll_clinch_atmpted,r_roll_clinch_acc,r_roll_ground_landed,r_roll_ground_atmpted,r_roll_ground_acc,r_roll_landed_head_per,r_roll_landed_body_per,r_roll_landed_leg_per,r_roll_landed_dist_per,r_roll_landed_clinch_per,r_roll_landed_ground_per,b_roll_kd,b_roll_sig_str_landed,b_roll_sig_str_atmpted,b_roll_sig_str_acc,b_roll_total_str_landed,b_roll_total_str_atmpted,b_roll_total_str_acc,b_roll_td_landed,b_roll_td_atmpted,b_roll_td_acc,b_roll_sub_att,b_roll_ctrl,b_roll_head_landed,b_roll_head_atmpted,b_roll_head_acc,b_roll_body_landed,b_roll_body_atmpted,b_roll_body_acc,b_roll_leg_landed,b_roll_leg_atmpted,b_roll_leg_acc,b_roll_dist_landed,b_roll_dist_atmpted,b_roll_dist_acc,b_roll_clinch_landed,b_roll_clinch_atmpted,b_roll_clinch_acc,b_roll_ground_landed,b_roll_ground_atmpted,b_roll_ground_acc,b_roll_landed_head_per,b_roll_landed_body_per,b_roll_landed_leg_per,b_roll_landed_dist_per,b_roll_landed_clinch_per,b_roll_landed_ground_per,target,r_stance_Orthodox,r_stance_Sideways,r_stance_Southpaw,r_stance_Switch,b_stance_Orthodox,b_stance_Sideways,b_stance_Southpaw,b_stance_Switch
0,180.34,70.31,185.42,5.05,55,4.19,42,4.35,37,70,1.6,180.34,70.31,190.50,3.84,57,4.16,60,0.00,0,84,0.0,0.444444,40.666667,72.777778,57.777778,61.222222,97.000000,65.555556,2.111111,5.333333,39.888889,0.777778,200.222222,23.111111,51.888889,47.666667,11.777778,14.888889,76.000000,5.777778,6.000000,87.111111,23.888889,47.555556,52.333333,8.000000,9.777778,60.555556,8.777778,15.444444,54.000000,53.444444,32.444444,12.888889,63.444444,17.666667,18.000000,1.000000,32.00,54.000000,55.000000,34.333333,56.333333,56.000000,0.000000,0.333333,0.00,0.000000,7.666667,26.000000,46.333333,57.000000,2.000000,3.000000,25.0,4.000000,4.666667,66.666667,27.666667,49.000000,53.000000,1.333333,1.333333,66.666667,3.000000,3.666667,59.333333,87.666667,2.666667,9.00,91.000000,2.666667,6.000000,1,0,0,1,0,1,0,0,0
1,190.50,83.91,190.50,4.28,54,3.17,58,0.74,32,81,1.0,185.42,83.91,190.50,3.44,56,2.50,60,1.28,48,76,0.4,0.181818,63.909091,114.909091,56.363636,81.181818,136.363636,60.727273,0.818182,2.545455,25.545455,1.090909,153.454545,43.727273,89.727273,50.909091,12.818182,16.727273,71.545455,7.363636,8.454545,77.909091,44.909091,90.272727,50.363636,10.636364,12.181818,78.000000,8.363636,12.454545,59.818182,69.818182,16.454545,12.545455,67.909091,15.272727,15.363636,0.428571,48.00,81.285714,58.857143,70.000000,108.714286,66.714286,1.285714,2.285714,40.00,0.571429,283.714286,32.428571,60.142857,52.857143,8.714286,13.142857,60.0,6.857143,8.000000,92.428571,37.714286,69.142857,51.285714,1.428571,2.000000,55.428571,8.857143,10.142857,66.428571,63.285714,20.142857,15.00,72.714286,4.000000,22.142857,1,1,0,0,0,0,0,1,0
2,190.50,92.99,193.04,3.32,43,3.93,51,0.30,66,80,0.1,190.50,92.99,193.04,2.50,45,3.04,43,1.43,18,35,1.3,0.100000,31.300000,73.100000,43.000000,36.400000,79.300000,45.700000,0.200000,0.300000,10.000000,0.100000,56.000000,22.300000,62.400000,34.400000,5.800000,7.300000,75.400000,3.200000,3.400000,86.800000,27.500000,67.400000,41.400000,3.100000,4.800000,50.800000,0.700000,0.900000,18.600000,65.300000,17.300000,16.000000,88.700000,8.800000,1.600000,0.050000,20.75,45.700000,50.350000,35.300000,62.200000,61.050000,0.800000,4.150000,25.05,0.750000,66.200000,10.550000,31.500000,36.600000,6.550000,9.950000,64.7,3.650000,4.250000,63.100000,1

In [ ]:
#df_model.to_csv("UFC_model_date.csv")

# Train-Test Split

In [13]:
X = df_model.drop(columns = ['target'])
y = df_model.target
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42, stratify = y)          

# LogReg

In [ ]:
# lr = LogisticRegression(max_iter=20000, random_state=10)

In [ ]:
# lr.fit(X_train, y_train)

In [ ]:
# X_test = X_test.copy()
# X_test[['prob_red', 'prob_blue']] = lr.predict_proba(X_test)
# X_test['y_pred'] = np.where(X_test['prob_red'] > 0.5, 1, 0)

In [ ]:
# # Results function from DT & RF notebook
# def get_results(actual, predicted):
#     print(f'The accuracy of your model is: {metrics.accuracy_score(actual, predicted)}')
#     print(f'The recall of your model is: {metrics.recall_score(actual, predicted)}')
#     print(f'The precision of your model is: {metrics.precision_score(actual, predicted)}')
#     print(f'The F1-score of your model is: {metrics.f1_score(actual, predicted)}')

In [ ]:
# get_results(y_test, X_test['y_pred'])

# Decision Tree

In [ ]:
# dt = DecisionTreeClassifier()

In [ ]:
# grid = GridSearchCV(estimator = DecisionTreeClassifier(),
#                     param_grid = {'max_depth': [3, 5, 7, 10], #4 different depths
#                                   'min_samples_split': [5, 10, 15, 20, 50, 100], #6 different split criteria
#                                   'min_samples_leaf': [2, 3, 4, 5, 6, 7], #6 different leaf sizes
#                                   'max_features':[2,5]}, #2 different features considered for each split
#                     cv = 10, # 10 folds for cross validation
#                     refit = True,
#                     verbose = 2,
#                     scoring = 'accuracy')

In [ ]:
# grid.fit(X_train, y_train)

In [ ]:
# grid.best_estimator

In [ ]:
# y_test_pred = grid.best_estimator.predict(X_test)

In [ ]:
# metrics.accuracy_score(y_test,y_test_pred)

# Random Forest

In [ ]:
# rf = RandomForestClassifier(n_estimators=50)

In [ ]:
# rf_params = {'max_depth': [3, 5, 7, 10], #4 different depths
#              'min_samples_split': [5, 10, 15, 20, 50, 100], #6 different split criteria
# #             'min_samples_leaf': [2, 3, 4, 5, 6, 7], #6 different leaf sizes
# #             'max_features':[2,5] #2 different features considered for each split
# }

# gs = GridSearchCV(rf, param_grid=rf_params, cv=5, scoring = 'accuracy')
# gs.fit(X_train, y_train)
# print(gs.best_score_)
# gs.best_params_

In [ ]:
# y_test_pred = gs.best_estimator.predict(X_test)

In [ ]:
# metrics.accuracy_score(y_test,y_test_pred)

# XGBoost

In [14]:
xgb_model = XGBClassifier(random_state = 42)

In [15]:
xgb_params = {"n_estimators": [200, 400],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1],
    "colsample_bytree": [0.8, 1]}
gs = GridSearchCV(xgb_model, param_grid=xgb_params, cv=5, scoring = 'accuracy')

In [16]:
xgb_params = {
    "n_estimators": [200, 400],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8],          
    "colsample_bytree": [0.8],   
    "min_child_weight": [1, 5],  
    "gamma": [0, 0.5],           
}
gs = GridSearchCV(xgb_model, param_grid=xgb_params, cv=5, scoring = 'accuracy')

In [17]:
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

0.7060839760068552


{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 3,
 'min_child_weight': 5,
 'n_estimators': 400,
 'subsample': 0.8}

In [18]:
best = gs.best_estimator_

In [19]:
y_test_pred = best.predict(X_test)

In [20]:
metrics.accuracy_score(y_test,y_test_pred)

0.6986410871302957

In [21]:
metrics.f1_score(y_test,y_test_pred)

0.7779740871613663

In [22]:
metrics.recall_score(y_test,y_test_pred)

0.8318639798488665

In [23]:
metrics.precision_score(y_test,y_test_pred)

0.730641592920354

In [24]:
metrics.log_loss(y_test,y_test_pred)

10.862076201196775

In [25]:
metrics.brier_score_loss(y_test,y_test_pred)

np.float64(0.30135891286970423)

In [ ]:
y_pred_proba = gs.best_estimator_.predict_proba(X_test)[:, 1]
y_pred_label = gs.best_estimator_.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_label))
print("Log loss:", log_loss(y_test, y_pred_proba))


Accuracy: 0.6986410871302957
Log loss: 0.550327240396413


# Creating Lookup Table

* This lookup table is created to be used by the Streamlit app

In [ ]:
import pandas as pd


df_model_date = pd.read_csv("UFC_model_date.csv")
df_model_date["date"] = pd.to_datetime(df_model_date["date"], errors="coerce")

# Identify feature columns (everything except target, ids, etc.)
drop_cols = ["target", "winner", "winner_id"]
feature_cols = [c for c in df_model_date.columns if c not in drop_cols]

# Split red/blue feature columns
r_feature_cols = [c for c in feature_cols if c.startswith("r_")]
b_feature_cols = [c for c in feature_cols if c.startswith("b_")]

# Build red fighter rows
red = df_model_date[["r_id", "date"] + r_feature_cols].copy()
red = red.rename(columns={"r_id": "fighter_id"})
# remove r_ prefix
red.columns = ["fighter_id", "date"] + [c[2:] for c in r_feature_cols]

# Build blue fighter rows
blue = df_model_date[["b_id", "date"] + b_feature_cols].copy()
blue = blue.rename(columns={"b_id": "fighter_id"})
blue.columns = ["fighter_id", "date"] + [c[2:] for c in b_feature_cols]

# Stack into one fighter-level table
fighters = pd.concat([red, blue], ignore_index=True)

# Sort by fighter, date
fighters = fighters.sort_values(["fighter_id", "date"])

# Take last row per fighter
fighters_latest = fighters.groupby("fighter_id").tail(1).reset_index(drop=True)


In [14]:
#fighters_latest.to_csv("fighter_profile_table.csv", index=False)

In [ ]:
ufc_raw = pd.read_csv("UFC.csv")

# build id → name mapping
id_name_map = pd.concat([
    ufc_raw[["r_id", "r_name"]].rename(columns={"r_id": "fighter_id", "r_name": "name"}),
    ufc_raw[["b_id", "b_name"]].rename(columns={"b_id": "fighter_id", "b_name": "name"})
]).drop_duplicates("fighter_id")

fighters_latest = fighters_latest.merge(id_name_map, on="fighter_id", how="left")
fighters_by_name = fighters_latest.set_index("name")


In [29]:
fighters_by_name.head()

,fighter_id,date,id,height,weight,reach,splm,str_acc,sapm,str_def,td_avg,td_avg_acc,td_def,sub_avg,roll_kd,roll_sig_str_landed,roll_sig_str_atmpted,roll_sig_str_acc,roll_total_str_landed,roll_total_str_atmpted,roll_total_str_acc,roll_td_landed,roll_td_atmpted,roll_td_acc,roll_sub_att,roll_ctrl,roll_head_landed,roll_head_atmpted,roll_head_acc,roll_body_landed,roll_body_atmpted,roll_body_acc,roll_leg_landed,roll_leg_atmpted,roll_leg_acc,roll_dist_landed,roll_dist_atmpted,roll_dist_acc,roll_clinch_landed,roll_clinch_atmpted,roll_clinch_acc,roll_ground_landed,roll_ground_atmpted,roll_ground_acc,roll_landed_head_per,roll_landed_body_per,roll_landed_leg_per,roll_landed_dist_per,roll_landed_clinch_per,roll_landed_ground_per,stance_Orthodox,stance_Sideways,stance_Southpaw,stance_Switch
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Mark Choinski,001eb2ab0f30e7ea,2025-06-07,001eb2ab0f30e7ea,172.72,70.31,175.26,3.87,50,7.33,33,2.00,18,0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1,0,0,0
Gabriel Silva,002ca196477ce572,2020-02-29,002ca196477ce572,167.64,61.23,180.34,1.30,34,3.47,45,1.50,25,38,0.5,0.0,8.000000,16.0,50.000000,58.000000,76.0,76.000000,1.000000,6.000000,16.0,0.0,266.0,7.000000,14.000000,50.0,0.0,1.000000,0.000000,1.000000,1.0,100.000000,6.000000,12.0,50.000000,2.000000,4.000000,50.000000,0.0,0.000000,0.0,87.0,0.0,12.000000,75.0,25.0,0.000000,1,0,0,0
Aalon Cruz,003d82fa384ca1d0,2021-03-06,003d82fa384ca1d0,182.88,65.77,198.12,7.58,39,8.88,58,0.00,0,0,0.0,0.0,2.000000,12.0,16.000000,2.000000,12.0,17.000000,0.000000,1.000000,0.0,0.0,5.0,2.000000,4.000000,50.0,0.0,4.000000,0.000000,0.000000,4.0,0.000000,2.000000,12.0,17.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,100.0,0.0,0.000000,100.0,0.0,0.000000,0,0,0,1
Davi Ramos,0052de90691d4a93,2020-07-18,0052de90691d4a93,167.64,70.31,177.80,2.64,37,3.44,56,1.95,40,83,0.6,0.0,28.333333,72.5,44.833333,36.666667,82.0,51.833333,1.666667,3.333333,49.5,0.5,123.0,21.166667,59.333333,43.0,3.5,9.166667,51.333333,3.666667,4.0,63.333333,21.666667,64.5,28.166667,2.666667,3.333333,44.833333,4.0,4.666667,55.5,74.5,15.5,9.333333,66.5,8.5,24.166667,1,0,0,0
Sean McCorkle,008dc37cca279def,2011-03-26,008dc37cca279def,200.66,120.20,205.74,0.85,44,2.48,52,2.12,40,33,4.2,0.0,2.000000,6.5,22.000000,4.000000,8.5,31.000000,0.500000,1.500000,50.0,2.0,94.0,2.000000,6.500000,22.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.5,0.000000,0.000000,0.000000,0.000000,2.0,4.000000,25.0,50.0,0.0,0.000000,0.0,0.0,50.000000,1,0,0,0


In [52]:
fighters_by_name.to_csv("fighters.csv")

In [ ]:
df = pd.read_csv("UFC.csv")
fighters = pd.read_csv("fighters.csv")

r_div = df[["r_name", "division"]].rename(columns={"r_name": "name"})
b_div = df[["b_name", "division"]].rename(columns={"b_name": "name"})

name_div = pd.concat([r_div, b_div], ignore_index=True)

name_div = name_div.drop_duplicates(subset=["name", "division"])

fighters_with_division = name_div.merge(fighters, on="name", how="left")

#fighters_with_division.to_csv("fighters_with_division.csv", index=False)


# Working

In [ ]:
df_fighters = pd.read_csv("fighters_with_division.csv")

required_cols = {"name", "division"}
missing = required_cols - set(df_fighters.columns)
if missing:
    raise ValueError(f"fighters.csv is missing required columns: {missing}")

df_fighters = df_fighters.set_index(["name", "division"])
df_fighters = df_fighters.sort_index()

feature_names = [
    'r_height',
    'r_weight',
    'r_reach',
    'r_splm',
    'r_str_acc',
    'r_sapm',
    'r_str_def',
    'r_td_avg',
    'r_td_avg_acc',
    'r_td_def',
    'r_sub_avg',
    'b_height',
    'b_weight',
    'b_reach',
    'b_splm',
    'b_str_acc',
    'b_sapm',
    'b_str_def',
    'b_td_avg',
    'b_td_avg_acc',
    'b_td_def',
    'b_sub_avg',
    'r_roll_kd',
    'r_roll_sig_str_landed',
    'r_roll_sig_str_atmpted',
    'r_roll_sig_str_acc',
    'r_roll_total_str_landed',
    'r_roll_total_str_atmpted',
    'r_roll_total_str_acc',
    'r_roll_td_landed',
    'r_roll_td_atmpted',
    'r_roll_td_acc',
    'r_roll_sub_att',
    'r_roll_ctrl',
    'r_roll_head_landed',
    'r_roll_head_atmpted',
    'r_roll_head_acc',
    'r_roll_body_landed',
    'r_roll_body_atmpted',
    'r_roll_body_acc',
    'r_roll_leg_landed',
    'r_roll_leg_atmpted',
    'r_roll_leg_acc',
    'r_roll_dist_landed',
    'r_roll_dist_atmpted',
    'r_roll_dist_acc',
    'r_roll_clinch_landed',
    'r_roll_clinch_atmpted',
    'r_roll_clinch_acc',
    'r_roll_ground_landed',
    'r_roll_ground_atmpted',
    'r_roll_ground_acc',
    'r_roll_landed_head_per',
    'r_roll_landed_body_per',
    'r_roll_landed_leg_per',
    'r_roll_landed_dist_per',
    'r_roll_landed_clinch_per',
    'r_roll_landed_ground_per',
    'b_roll_kd',
    'b_roll_sig_str_landed',
    'b_roll_sig_str_atmpted',
    'b_roll_sig_str_acc',
    'b_roll_total_str_landed',
    'b_roll_total_str_atmpted',
    'b_roll_total_str_acc',
    'b_roll_td_landed',
    'b_roll_td_atmpted',
    'b_roll_td_acc',
    'b_roll_sub_att',
    'b_roll_ctrl',
    'b_roll_head_landed',
    'b_roll_head_atmpted',
    'b_roll_head_acc',
    'b_roll_body_landed',
    'b_roll_body_atmpted',
    'b_roll_body_acc',
    'b_roll_leg_landed',
    'b_roll_leg_atmpted',
    'b_roll_leg_acc',
    'b_roll_dist_landed',
    'b_roll_dist_atmpted',
    'b_roll_dist_acc',
    'b_roll_clinch_landed',
    'b_roll_clinch_atmpted',
    'b_roll_clinch_acc',
    'b_roll_ground_landed',
    'b_roll_ground_atmpted',
    'b_roll_ground_acc',
    'b_roll_landed_head_per',
    'b_roll_landed_body_per',
    'b_roll_landed_leg_per',
    'b_roll_landed_dist_per',
    'b_roll_landed_clinch_per',
    'b_roll_landed_ground_per',
    'r_stance_Orthodox',
    'r_stance_Sideways',
    'r_stance_Southpaw',
    'r_stance_Switch',
    'b_stance_Orthodox',
    'b_stance_Sideways',
    'b_stance_Southpaw',
    'b_stance_Switch'
]


def get_fighter_row(name: str, division: str) -> pd.Series:
    """Safely fetch a fighter row for a given (name, division) with good errors."""
    idx = (name, division)
    if idx not in df_fighters.index:
        # Check if the fighter exists in any division at all
        if name not in df_fighters.index.get_level_values("name"):
            raise ValueError(f"Fighter '{name}' not found in fighters.csv.")
        else:
            # Fighter exists but not in this division
            available_divs = df_fighters.loc[name].index.unique().tolist()
            raise ValueError(
                f"Fighter '{name}' has no data in division '{division}'. "
                f"Available divisions: {available_divs}"
            )
    row = df_fighters.loc[idx]
    # If accidentally multiple rows, take the most recent or first;
    # here we just take the first if it's a DataFrame
    if isinstance(row, pd.DataFrame):
        row = row.iloc[0]
    return row


def build_match_row(
    fighter_red: str,
    division_red: str,
    fighter_blue: str,
    division_blue: str
) -> pd.DataFrame:
    """
    Create a single-row dataframe with the same columns as the model expects,
    for a given (red_fighter, red_division) vs (blue_fighter, blue_division).
    """

    # Ensure same division (or decide your own rule here)
    if division_red != division_blue:
        raise ValueError(
            f"Division mismatch: '{fighter_red}' is in '{division_red}', "
            f"'{fighter_blue}' is in '{division_blue}'. "
            "Fights should be within the same division."
        )

    # Get fighter rows
    r = get_fighter_row(fighter_red, division_red)
    b = get_fighter_row(fighter_blue, division_blue)

    # Build feature row
    row = {}
    for col in feature_names:
        if col.startswith("r_"):
            base = col[2:]  # strip leading 'r_'
            row[col] = r.get(base, np.nan)
        elif col.startswith("b_"):
            base = col[2:]  # strip leading 'b_'
            row[col] = b.get(base, np.nan)
        else:
            # For any non-prefixed feature (you don't have any here, but just in case)
            row[col] = 0

    X_input = pd.DataFrame([row])
    # Ensure correct column order
    X_input = X_input[feature_names]
    # Handle any leftover NaNs (should be rare)
    X_input = X_input.fillna(0)
    return X_input


def predict_match(
    fighter_red: str,
    division_red: str,
    fighter_blue: str,
    division_blue: str
):
    """
    Predict the winner between (fighter_red, division_red)
    and (fighter_blue, division_blue).
    """
    X_input = build_match_row(fighter_red, division_red, fighter_blue, division_blue)
    proba_red = best.predict_proba(X_input)[0, 1]  # P(red wins)

    if proba_red >= 0.5:
        winner = fighter_red
        confidence = proba_red
    else:
        winner = fighter_blue
        confidence = 1 - proba_red

    return winner, confidence

fighter1 = str(input("Please enter the first fighter's name: "))
fighter2 = str(input("Please enter the second fighter's name: "))
div = str(input("Please enter the division: ").lower())

winner, conf = predict_match(fighter1, div, fighter2, div)
print(f"Predicted winner: {winner} (confidence: {conf:.2%})")

Predicted winner: Jon Jones (confidence: 86.92%)


In [31]:
from pprint import pprint
divisions = df_fighters.index.get_level_values("division").unique()
pprint(divisions)


Index(['bantamweight', 'lightweight', 'middleweight', 'welterweight',
       'featherweight', 'heavyweight', 'light heavyweight',
       'women's bantamweight', 'women's featherweight', 'women's strawweight',
       ...
       'ultimate fighter brazil 3 middleweight tournament', '7 tournament',
       'ultimate fighter 4 middleweight tournament', '2 tournament',
       'ultimate fighter 13 welterweight tournament',
       '13 heavyweight tournament', 'road to  1 bantamweight tournament',
       'ultimate fighter 32 middleweight tournament',
       'ultimate fighter 8 light heavyweight tournament',
       'ultimate fighter china welterweight tournament'],
      dtype='object', name='division', length=107)


* Cleaning divisions due to having over 100 initially

In [ ]:
import pandas as pd

# 0. Load fighters_with_division.csv
df_fighters = pd.read_csv("fighters_with_division.csv")

# 1. Make sure 'name' and 'division' are normal columns
#    (in case you previously set them as index)
if "name" not in df_fighters.columns or "division" not in df_fighters.columns:
    df_fighters = df_fighters.reset_index()

# 2. Ensure 'division' is a string column
df_fighters["division"] = df_fighters["division"].astype(str)

# 3. Lowercase for easier matching
df_fighters["division_lower"] = df_fighters["division"].str.lower()

# 4. Regex to extract only the divisions you care about
pattern = (
    r"(women's strawweight|women's flyweight|women's bantamweight|women's featherweight|"
    r"flyweight|bantamweight|featherweight|lightweight|welterweight|middleweight|light heavyweight|heavyweight)"
)

df_fighters["division_clean"] = df_fighters["division_lower"].str.extract(
    pattern, expand=False
)

# 5. Drop any rows where we *couldn't* extract a clean division
df_fighters = df_fighters.dropna(subset=["division_clean"])

# 6. Nicely title-case the final division names
df_fighters["division_clean"] = df_fighters["division_clean"].str.title()

# 7. Rename to just 'division' again
df_fighters = df_fighters.drop(columns=["division_lower", "division"])
df_fighters = df_fighters.rename(columns={"division_clean": "division"})

# 8. Put (name, division) back as the MultiIndex (for your prediction function)
df_fighters = df_fighters.set_index(["name", "division"]).sort_index()

print(df_fighters.index.get_level_values("division").unique())
df_fighters.shape

Index(['Bantamweight', 'Lightweight', 'Middleweight', 'Welterweight',
       'Featherweight', 'Heavyweight', 'Light Heavyweight',
       'Women'S Bantamweight', 'Women'S Featherweight', 'Women'S Strawweight',
       'Flyweight', 'Women'S Flyweight'],
      dtype='object', name='division')


(3517, 54)

In [ ]:
df_fighters.to_csv("fighters_lookup.csv")